In [8]:
j = {
    1: [2,3],
    2: [3,4,5],
    3: [4],
    4: [5,6],
    5: [],
    6: []
}
unique_bs = set()
info = {k: None for k in j.keys()}
b_names = list(j.keys())
for b_name in b_names:
    if not b_name in j:
        continue
    unique_bs.add(b_name)
    pending_bs = set(j[b_name])
    pending_bs.add(b_name)
    del j[b_name]
    print(pending_bs)
    temp_bs = set()
    while pending_bs:
        for similar_b in pending_bs:
            print(similar_b)
            if not similar_b in j:
                continue
            if info[similar_b] is None:
                info[similar_b] = b_name
            temp_bs.update(j[similar_b])
            del j[similar_b]
        print(pending_bs)
        pending_bs = temp_bs
        temp_bs = set()
print(unique_bs)
print(info)

{1, 2, 3}
1
2
3
{1, 2, 3}
3
4
5
{3, 4, 5}
5
6
{5, 6}
{1}
{1: None, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1}


In [5]:
mapping = {
    1: [2,3],
    2: [3,4,5],
    3: [4],
    4: [5,6],
    5: [],
    6: []
}


info = {k: None for k in mapping}
sorted_mappings = sorted([(k, v) for k, v in mapping.items()],
                         key=lambda x: len(x[1]),
                         reverse=True)

for k, v in sorted_mappings:
    for dup in v:
        if info[dup] is None:
            info[dup] = k

unique_bs = [k for k in info if info[k] is None]
unique_bs.extend([v for v in info.values() if v is not None])
unique_bs = set(unique_bs)

print(unique_bs)

{1, 2, 4}


In [10]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import applications
import tensorflow.keras.applications.vgg16 as vgg16

In [25]:
def resize_image(image, shape = (224,224)):
  target_width = shape[0]
  target_height = shape[1]
  initial_width = tf.shape(image)[0]
  initial_height = tf.shape(image)[1]
  im = image
  ratio = 0
  if(initial_width < initial_height):
    ratio = tf.cast(256 / initial_width, tf.float32)
    h = tf.cast(initial_height, tf.float32) * ratio
    im = tf.image.resize(im, (256, h), method="bicubic")
  else:
    ratio = tf.cast(256 / initial_height, tf.float32)
    w = tf.cast(initial_width, tf.float32) * ratio
    im = tf.image.resize(im, (w, 256), method="bicubic")
  width = tf.shape(im)[0]
  height = tf.shape(im)[1]
  startx = width//2 - (target_width//2)
  starty = height//2 - (target_height//2)
  im = tf.image.crop_to_bounding_box(im, startx, starty, target_width, target_height)
  return im

def scale16(image, label):
  i = image
  i = tf.cast(i, tf.float32)
  i = resize_image(i, (224,224))
  i = vgg16.preprocess_input(i)
  return (i, label)

def batch_set(dataset, batch_size):
    return dataset.map(scale16) \
                  .shuffle(1000) \
                  .batch(batch_size) \
                  .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

def create_batched_datasets(batch_size = 64):
    datasets, info = tfds.load(name="imagenet_v2", 
                            with_info=True, 
                            as_supervised=True,
                            )
#     train = batch_set(datasets['train'], batch_size)
    val = batch_set(datasets['test'], batch_size)
    return val, info

In [26]:
test_dataset, info = create_batched_datasets()

model = vgg16.VGG16(weights='imagenet', include_top=True)

model.compile('sgd', 'categorical_crossentropy', 
              ['sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])

model.evaluate(test_dataset)

553467904/553467096 [==============================] - 50s 0us/step


ValueError: in user code:

    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1208 run_step  **
        outputs = model